In [1]:
import sys
sys.path.insert(0, "../")
from pathlib import Path

from tqdm import tqdm

from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration
import torch
from torchvision.transforms.functional import pil_to_tensor
from PIL import Image
import requests
from artstract_ml.dataset import load_image_dataset
from more_itertools import chunked

/home/nicolas/.conda/envs/mosaic/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import json

with open("/home/nicolas/projects/artstract-ml/dataset/caption_data.json") as f:
  out = json.load(f)

In [3]:
model = InstructBlipForConditionalGeneration.from_pretrained("Salesforce/instructblip-vicuna-7b", load_in_4bit=True)
model.config.text_config.pad_token_id = 0
processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-vicuna-7b", load_in_4bit=True)

Loading checkpoint shards: 100%|██████████| 4/4 [00:14<00:00,  3.55s/it]
Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
t = lambda img: processor.image_processor(image)["pixel_values"][0]
_, _, test = load_image_dataset(Path("../dataset/"), transform=lambda i: pil_to_tensor(i.convert("RGB")))

In [5]:
true = []
preds = []

PROMPT = "%s What abstract concept depicts this image among ['comfort', 'danger', 'death', 'fitness', 'freedom', 'power', 'safety']?\nconcept: "
BATCH_SIZE = 4

for batch_idx, batch in tqdm(enumerate(chunked(test, BATCH_SIZE)), total=len(test) / BATCH_SIZE):
  start_idx = batch_idx * BATCH_SIZE
  end_idx = start_idx + BATCH_SIZE
  captions = [out[test.imgs[i][0].split("/")[-1].split(".")[0]]["description_w_strength"] for i in range(start_idx, end_idx)]
  img, label_idx = zip(*batch)
  true.extend([test.classes[l] for l in label_idx])
  
  prompts = [PROMPT % c for c in captions]
  
  inputs = processor(images=img, text=prompts, return_tensors="pt", truncation=True, padding=True).to(model.device)

  outputs = model.generate(
    **inputs,
    do_sample=True,
    max_new_tokens=20,
    top_p=0.9,
    repetition_penalty=1.5,
    length_penalty=1.0,
    temperature=1)

  decoded_preds = processor.batch_decode(outputs, skip_special_tokens=True)
  preds.extend([p.lower().strip() for p in decoded_preds])

  0%|          | 0/93.25 [00:00<?, ?it/s]/home/nicolas/.conda/envs/mosaic/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')
/home/nicolas/.conda/envs/mosaic/lib/python3.11/site-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
 12%|█▏        | 11/93.25 [01:17<09:38,  7.04s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB (GPU 0; 23.68 GiB total capacity; 14.57 GiB already allocated; 180.44 MiB free; 16.44 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
import re
import random

clusters_re = re.compile("danger|comfort|freedom|death|fitness|power|safety")

In [ ]:
preds_corr = []

for p in preds:
  re_result = clusters_re.findall(p)

  if len(re_result) > 0:
    preds_corr.append(re_result[0])
  else:
    preds_corr.append(random.choice(["danger","comfort","freedom","death","fitness","power","safety"]))

In [ ]:
print(classification_report(true, preds_corr))

              precision    recall  f1-score   support

     comfort       0.40      0.04      0.07       603
      danger       0.11      0.08      0.09       170
       death       0.25      0.42      0.31       257
     fitness       0.12      0.05      0.07       102
     freedom       0.06      0.47      0.10        51
       power       0.24      0.31      0.27       276
      safety       0.04      0.06      0.04        33

    accuracy                           0.18      1492
   macro avg       0.17      0.20      0.14      1492
weighted avg       0.27      0.18      0.15      1492



In [ ]:
with open("output.txt", "w") as f:
  json.dump(preds, f)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(true, preds))

                                                                                                   precision    recall  f1-score   support

                                                                                                        0.00      0.00      0.00         0
                                                                                        "comfort"       0.00      0.00      0.00         0
                                                                                          "danger       0.00      0.00      0.00         0
                                                                                         "danger"       0.00      0.00      0.00         0
                                                                                          "death"       0.00      0.00      0.00         0
                                                                                        "fitness"       0.00      0.00      0.00         0
                          

/home/nicolas/.conda/envs/mosaic/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/nicolas/.conda/envs/mosaic/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/nicolas/.conda/envs/mosaic/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/